In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join

In [2]:
def load_abs_surfs():
    abs_surfs = pd.read_excel('data/absSurfs.xlsx').set_index("Site")
    abs_surfs = abs_surfs.stack()
    abs_surfs.index.names = ["Site", "CLC"]
    idx = abs_surfs.index
    abs_surfs.index = abs_surfs.index.set_levels([idx.levels[0].to_series(), idx.levels[1].to_series().astype(int)])
    abs_surfs = pd.DataFrame({'surface': abs_surfs})
    return abs_surfs

In [3]:
def load_pesticides():
    pesticides = pd.read_excel('data/pesticides.xlsx').rename(columns={'importName': 'pesticide'})
    pesticides = pesticides.set_index("pesticide")
    pesticides["typeEN"] = pesticides["typeEN"].str.lower().str.replace("and", ",",regex=True).str.replace(" +", "",regex=True)
    pesticides["familyEN"] = pesticides["familyEN"].str.lower().str.replace(" +", ",",regex=True)
    return pesticides

In [4]:
def load_libelles():
    a = pd.read_excel('data/clc-nomenclature-c.xls', sheet_name="nomenclature_clc_1").rename(columns={'code_clc_1': "CLC"})
    b = pd.read_excel('data/clc-nomenclature-c.xls', sheet_name="nomenclature_clc_2").rename(columns={'code_clc_2': "CLC"})
    c = pd.read_excel('data/clc-nomenclature-c.xls', sheet_name="nomenclature_clc_3").rename(columns={'code_clc_3': "CLC"})
    libelles = pd.concat((a, b, c)).set_index("CLC")
    return libelles

In [5]:
def load_disthive():
    dist_beehive = pd.read_excel('data/distsOneSheet.xlsx').rename(columns={"classCLC": "CLC"}).set_index(["Site", "polyID", "CLC"])
    return dist_beehive

In [6]:
def load_periods(category):
    if category not in ["HM", "Pesticides"]:
        raise ValueError("cateogry can only be HM or Pesticides")
    
    all_data = []
    for year in ["2017", "2018", "2019", "2020"]:
        year_path = f"data/{year}/{category}"
        onlyfiles = [join(year_path, f) for f in listdir(year_path) if isfile(join(year_path, f)) and f.endswith(".xlsx") and not f.startswith('~')]
        for excel in onlyfiles:
            all_data.append(pd.read_excel(excel))
    return pd.concat(all_data).rename(columns={"REF....SUBSTANCE": "Site", "PERIOD": "Period"}).set_index(["Site", "Period"])

In [7]:
def load_heavy_metal_lmr():
    lmr = pd.read_csv("data/LMR.txt", sep="\t").drop("Unnamed: 7", axis=1).stack().droplevel(0)
    lmr.index.name = "heavymetal"
    lmr = pd.DataFrame({'LMR': lmr})
    return lmr

In [8]:
def get_pesticides_flags(phm_grouped, pesticide_cat, pesticides_family):
    df = phm_grouped['pesticide'].stack()
    # Pesticides
    df.index.names = ["Site", "pesticide"]
    site_per_pesticide = df.reset_index().rename(columns={0: 'level'}).set_index('pesticide').merge(pesticides[['LMR']], left_index=True, right_index=True)
    site_per_pesticide = site_per_pesticide.assign(
        above_LMR=site_per_pesticide['level'] > site_per_pesticide['LMR'],
        present=site_per_pesticide['level'] > 0
    )

    pesticide_cat_per_site = (site_per_pesticide.merge(pesticide_cat, left_index=True, right_index=True).groupby(['Site', 'pesticide_cat']).max())
    pesticide_cat_per_site = pesticide_cat_per_site.unstack()

    pesticide_fam_per_site = (site_per_pesticide.merge(pesticides_family, left_index=True, right_index=True).groupby(['Site', 'pesticide_family']).max())
    pesticide_fam_per_site = pesticide_fam_per_site.unstack()

    site_per_pesticide = site_per_pesticide.reset_index().set_index(['Site', 'pesticide'])

    pesticides_flags = pd.concat(
        [
            site_per_pesticide.unstack(),
            pesticide_cat_per_site,
            pesticide_fam_per_site,
        ],
        axis=1,
        keys=['pesticide', 'pesticide_cat','pesticide_family'],
    )
    pesticides_flags

    return pesticides_flags

In [9]:
def get_heavymetal_flags(phm_grouped):
    df = phm_grouped['heavymetal'].stack()
    # Pesticides
    df.index.names = ["Site", "heavymetal"]
    df = df.reset_index().rename(columns={0: 'level'}).set_index('heavymetal')
    df = df.merge(heavy_metal_lmr, left_index=True, right_index=True).reset_index().set_index(['Site', 'heavymetal'])
    df = df.assign(
        above_LMR=df['level'] > df['LMR'],
        present=df['level'] > 0
    )
    df = df.unstack()
    # Add column level
    df = pd.concat([df], axis=1, keys=['heavymetal'])
    return df

In [10]:
def get_phm_flags(phm_grouped, pesticide_cat, pesticides_family):
    df = pd.concat([
        get_pesticides_flags(phm_grouped, pesticide_cat, pesticides_family),
        get_heavymetal_flags(phm_grouped),
    ], axis=1)
    # Drop LMR as it's not useful
    df = df.drop(['LMR'], axis=1, level=1)
    return df

In [11]:
abs_surfs = load_abs_surfs()
abs_surfs.head(10)

surface
Site  CLC              
F0069 111  0.000000e+00
      112  1.064122e+06
      121  3.587087e+05
      122  0.000000e+00
      123  0.000000e+00
      124  0.000000e+00
      131  0.000000e+00
      132  0.000000e+00
      133  0.000000e+00
      141  0.000000e+00

In [12]:
pesticides = load_pesticides()
pesticides.head(10)

,name,statusEN,LMR,familyEN,typeEN
pesticide,,,,,
Aclonifen,aclonifen,authorized,0.05,"diphenyl,ether",herbicide
Aldrin,aldrin,prohibited,0.01,organochlorine,insecticide
Ametoctradin,ametoctradin,authorized,0.05,triazolopyrimidine,fungicide
Anthraquinone,anthraquinone,prohibited,0.02,"polycyclic,aromatic,hydrocarbon",birdrepellant
Azoxystrobin,azoxystrobin,authorized,0.05,strobilurin,fungicide
Benalaxyl,benalaxyl,prohibited,0.05,phenylamides,fungicide
Benfluralin,benfluralin,authorized,0.05,dinitroaniline,herbicide
Benthiavalicarb.iso,benthiavalicarb-isopropyl,authorized,0.05,carbamate,fungicide
Benzovindiflupyr,benzovindiflupyr,authorized,0.05,amide,fungicide


In [13]:
heavy_metal_lmr = load_heavy_metal_lmr()
heavy_metal_lmr.head(10)

,LMR
heavymetal,
Lead,1.5
Cadmium,1.0
Arsenic,0.3
Mercury,1.0
Copper,30.0


In [14]:
libelles = load_libelles()
libelles.head(10)

,libelle_en
CLC,
1,Artificial surfaces
2,Agricultural areas
3,Forest and semi natural areas
4,Wetlands
5,Water bodies
11,Urban fabric
12,"Industrial, commercial and transport units"
13,"Mine, dump and construction sites"
14,"Artificial, non-agricultural vegetated areas"


In [15]:
disthive = load_disthive()
disthive.head(10)

dist
Site  polyID    CLC             
F0069 FR-88090  231  1183.022362
      FR-131773 242   289.926324
      FR-46167  211   221.344153
      FR-11047  112  1038.383069
      FR-195343 311   724.045683
      FR-34698  142  1094.991277
      FR-46111  211   258.423271
      FR-27481  121     0.000000
      FR-11011  112   788.720024
      FR-87971  231  1224.227665

In [16]:
heavy_metal_periods = load_periods("HM")
heavy_metal_periods.head(10)

,,Lead,Cadmium,Arsenic,Mercury,Copper,Chromium,Zinc,Chrome
Site,Period,,,,,,,,
B0017,2017/P1,0.081,0.810,0.0,0.0,5.5,0.032,40,NaN
B0019,2017/P1,0.062,0.360,0.0,0.0,8.5,0.029,33,NaN
B0021,2017/P1,0.150,0.610,0.0,0.0,7.0,0.038,37,NaN
B0023,2017/P1,0.067,0.350,0.0,0.0,6.9,0.031,37,NaN
B0037,2017/P1,0.120,0.070,0.0,0.0,5.6,0.052,35,NaN
B0039,2017/P1,0.280,0.034,0.0,0.0,5.5,0.067,38,NaN
B0041,2017/P1,0.220,0.096,0.0,0.0,5.6,0.083,33,NaN
B0043,2017/P1,0.150,0.058,0.0,0.0,6.5,0.061,32,NaN
B0045,2017/P1,0.270,0.120,0.0,0.0,8.1,0.120,41,NaN


In [17]:
pesticides_periods = load_periods("Pesticides")
pesticides_periods.head(10)

,,Aldrin,Benalaxyl,Chlorpropham,Cyanofenphos,Cypermethrin,Ethofumesate,Fludioxonil,Fluopyram,Flutolanil,Pencycuron,...,Nicosulfuron,Piperonylbutoxide,Prosulfuron,Simazine,Terbuthylazine,Dieldrin,Fenamidone,Trifluralin,Cycloate,Pentachloroanisol
Site,Period,,,,,,,,,,,,,,,,,,,,,
B0017,2017/P1,0.0,0.0000,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0019,2017/P1,0.0,0.0188,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0021,2017/P1,0.0,0.0711,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0023,2017/P1,0.0,0.0294,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0037,2017/P1,0.0,0.0000,0.01,0.0,0.00,0.0,0.00,0.0,0.0,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0039,2017/P1,0.0,0.0000,0.00,0.0,0.01,0.0,0.00,0.0,0.0,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0041,2017/P1,0.0,0.0000,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0043,2017/P1,0.0,0.0000,0.00,0.0,0.00,0.0,0.01,0.0,0.0,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0045,2017/P1,0.0,0.0000,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
distsurf = disthive.groupby(['Site', 'CLC']).mean().merge(abs_surfs, left_index=True, right_index=True)

# Load historical data
phm = pd.concat(
    [pesticides_periods,
    heavy_metal_periods],
    axis=1,
    keys=['pesticide','heavymetal'],
)

# Taking the max of the historical measurements
phm_grouped = phm.groupby('Site').max().fillna(0)

# Categorize pesticides
pesticides_family = pd.DataFrame({'pesticide_family': pesticides[["familyEN"]].fillna("UNKNOWN").apply(lambda r: r[0].split(','), axis=1).explode()})
pesticide_cat = pd.DataFrame({'pesticide_cat': pesticides[["typeEN"]].fillna("UNKNOWN").apply(lambda r: r[0].split(','), axis=1).explode()})

# Pesticide and heavy metals flags
phm_flags = get_phm_flags(phm_grouped, pesticide_cat, pesticides_family)

# We've now got basic features and basic things to predict.
# Next steps are to engineer these features, have a look at them and select a subset from the things to predict
features = distsurf.unstack().fillna(0)

# Contains the level, above_LMR, present for the pesticides, pesticides_category, pesticides_family, heavy_metal
to_predict = phm_flags

print("Features categories: " + str(features.columns.get_level_values(0).unique()))

print("To predict categories: " + str(to_predict.columns.get_level_values(0).unique()))
print("To predict subcategories: " + str(to_predict.columns.get_level_values(1).unique()))

display(to_predict)
display(features)

Features categories: Index(['dist', 'surface'], dtype='object')
To predict categories: Index(['pesticide', 'pesticide_cat', 'pesticide_family', 'heavymetal'], dtype='object')
To predict subcategories: Index(['level', 'above_LMR', 'present'], dtype='object')


pesticide                                                           \
           level                                                            
       Aclonifen Aldrin Ametoctradin Anthraquinone Azoxystrobin Benalaxyl   
Site                                                                        
B0012     0.0048   0.00       0.0000        0.0000       0.0000    0.0000   
B0017     0.0000   0.00       0.0000        0.0000       0.0000    0.1850   
B0019     0.0000   0.00       0.0000        0.0000       0.0000    0.0259   
B0021     0.0000   0.00       0.0000        0.0000       0.0000    0.0757   
B0023     0.0000   0.00       0.0000        0.0036       0.0000    0.0294   
B0032     0.0056   0.00       0.0000        0.0000       0.0000    0.0590   
B0037     0.0043   0.00       0.0000        0.0039       0.0000    0.0000   
B0039     0.0037   0.00       0.0000        0.0000       0.0000    0.0070   
B0041     0.0052   0.00       0.0000        0.0000       0.0000    0.0248   
B0043     0.0120   0.00       0.0000        0.0050       0.0000    0.0061   
B0045     0.0000   0.00       0.0000        0.0000       0.0000    0.0000   
B0059     0.0000   0.00       0.0000        0.0000       0.0000    0.0230   
B0061     0.0000   0.00       0.0000        0.0000       0.0000    0.0388   
B0067     0.0054   0.00       0.0000        0.0045       0.0000    0.0118   
B0069     0.0030   0.00       0.0000        0.0070       0.0000    0.0086   
B0077     0.0072   0.00       0.0000        0.0047       0.0000    0.0423   
B0093     0.0000   0.00       0.0000        0.0000       0.0000    0.1329   
B0096     0.0000   0.00       0.0000        0.0000       0.0000    0.0000   
B0103     0.0093   0.00       0.0000        0.0061       0.0870    0.0560   
B0105     0.0180   0.00       0.0000        0.0100       0.0000    0.0102   
B0116     0.0053   0.00       0.0000        0.0000       0.0000    0.0000   
B0128     0.0000   0.00       0.0000        0.0000       0.0000    0.0000   
B0140     0.0048   0.00       0.0000        0.0077       0.0000    0.0000   
B0142     0.0000   0.00       0.0000        0.0038       0.0000    0.0054   
B0152     0.0070   0.00       0.0000        0.0000       0.0000    0.0468   
B0158     0.0000   0.00       0.0000        0.0034       0.0000    0.0984   
B0163     0.0000   0.00       0.0000        0.0139       0.0000    0.0230   
F0001     0.0000   0.00       0.9032        0.0000       0.0066    0.0050   
F0003     0.0000   0.00       0.0097        0.0000       0.0000    0.0000   
F0005     0.0000   0.00       0.0000        0.0000       0.0000    0.0221   
F0007     0.0000   0.00       0.0000        0.0000       0.0000    0.0101   
F0009     0.0000   0.00       0.0000        0.0000       0.0000    0.0335   
F0011     0.0000   0.01       0.0000        0.0000       0.0000    0.0000   
F0013     0.0000   0.00       0.0000        0.0000       0.0000    0.0407   
F0015     0.0000   0.00       0.0000        0.0000       0.0000    0.1399   
F0017     0.0000   0.00       0.0000        0.0071       0.0000    0.0034   
F0019     0.0000   0.00       0.0000        0.0000       0.0000    0.0000   
F0020     0.0000   0.00       0.0000        0.0000       0.0000    0.0640   
F0021     0.0000   0.00       0.0055        0.0039       0.0000    0.0222   
F0023     0.0000   0.00       0.0000        0.0000       0.0000    0.0477   
F0025     0.0050   0.00       0.0000        0.0051       0.0000    0.0194   
F0027     0.0000   0.00       0.0000        0.0000       0.0000    0.0000   
F0029     0.0070   0.00       0.0000        0.0000       0.1166    0.0034   
F0031     0.0000   0.00       0.0000        0.0000       0.0085    0.0000   
F0033     0.0000   0.00       0.0127        0.0000       0.0000    0.0000   
F0035     0.0000   0.00       0.0000        0.0000       0.0000    0.0287   
F0037     0.0000   0.00       0.0000        0.0000       0.0000    0.0000   
F0039     0.0000   0.00       0.0000        0.0000       0.0000    0.0587   
F00

dist                                                  \
CLC             111         112          121          122  123  124   
Site                                                                  
B0012   1020.033273  564.609800   117.220148     0.000000  0.0  0.0   
B0017      0.000000  230.733942  1457.209438     0.000000  0.0  0.0   
B0019      0.000000    0.000000     0.000000     0.000000  0.0  0.0   
B0021      0.000000    0.000000     0.000000     0.000000  0.0  0.0   
B0023      0.000000    0.000000     0.000000     0.000000  0.0  0.0   
...             ...         ...          ...          ...  ...  ...   
F0069      0.000000  737.549842     0.000000     0.000000  0.0  0.0   
F0071      0.000000    0.000000   990.596850     0.000000  0.0  0.0   
F0073      0.000000    0.000000     0.000000     0.000000  0.0  0.0   
F0075      0.000000  751.533157   344.517523  1444.909224  0.0  0.0   
FE0001     0.000000    0.000000     0.000000     0.000000  0.0  0.0   

                                           ...       surface                \
CLC            131  133  141          142  ...           312           313   
Site                                       ...                               
B0012      0.00000  0.0  0.0     0.000000  ...  0.000000e+00  0.000000e+00   
B0017      0.00000  0.0  0.0     0.000000  ...  1.097484e+06  1.871540e+06   
B0019      0.00000  0.0  0.0     0.000000  ...  5.146666e+06  1.330174e+06   
B0021      0.00000  0.0  0.0     0.000000  ...  1.572156e+06  3.620488e+06   
B0023      0.00000  0.0  0.0  1223.368654  ...  2.538647e+06  6.865176e+04   
...            ...  ...  ...          ...  ...           ...           ...   
F0069      0.00000  0.0  0.0  1094.991277  ...  0.000000e+00  0.000000e+00   
F0071   1066.57633  0.0  0.0     0.000000  ...  4.684407e+05  0.000000e+00   
F0073      0.00000  0.0  0.0     0.000000  ...  4.420773e+06  6.970899e+03   
F0075      0.00000  0.0  0.0     0.000000  ...  0.000000e+00  0.000000e+00   
FE0001     0.00000  0.0  0.0     0.000000  ...  0.000000e+00  0.000000e+00   

                                                                  \
CLC     322           324  331           412  423            511   
Site                                                               
B0012   0.0  0.000000e+00  0.0  0.000000e+00  0.0  486328.326111   
B0017   0.0  0.000000e+00  0.0  0.000000e+00  0.0       0.000000   
B0019   0.0  0.000000e+00  0.0  3.786370e+05  0.0       0.000000   
B0021   0.0  0.000000e+00  0.0  9.421463e+05  0.0       0.000000   
B0023   0.0  6.979889e+05  0.0  1.809062e+06  0.0       0.000000   
...     ...           ...  ...           ...  ...            ...   
F0069   0.0  0.000000e+00  0.0  0.000000e+00  0.0       0.000000   
F0071   0.0  7.933020e+05  0.0  0.000000e+00  0.0       0.000000   
F0073   0.0  1.621549e+06  0.0  0.000000e+00  0.0       0.000000   
F0075   0.0  0.000000e+00  0.0  0.000000e+00  0.0       0.000000   
FE0001  0.0  0.000000e+00  0.0  0.000000e+00  0.0       0.000000   

                            
CLC               512  523  
Site                        
B0012        0.000000  0.0  
B0017        0.000000  0.0  
B0019        0.000000  0.0  
B0021        0.000000  0.0  
B0023        0.000000  0.0  
...               ...  ...  
F0069        0.000000  0.0  
F0071   643979.018311  0.0  
F0073        0.000000  0.0  
F0075        0.000000  0.0  
FE0001       0.000000  0.0  

[63 rows x 52 columns]

In [19]:
to_predict.to_csv("to_predict.csv", index=False)